<a href="https://colab.research.google.com/github/truongcan0512/Image-Processing/blob/main/My_simplicity_bow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import os



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_dir = '/content/drive/MyDrive/Data/Simplycity'

In [4]:
impath= os.listdir(data_dir)

In [5]:
impath

['7', '6', '0', '4', '3', '9', '2', '5', '8', '1']

In [ ]:
!pip install opencv-python==3.4.2.17
!pip install opencv-contrib-python==3.4.2.17

In [7]:
import cv2
import numpy as np

In [8]:
#Read images from database:

In [9]:
# Tao list images va list id
train_image_list=[]
train_id=[]
test_image_list=[]
test_id=[]
for path in impath:
    class_path = data_dir + "/" +path
    image_names= os.listdir(class_path)
    image_path=[]
    image_id=[]
    for image in image_names:
        image_path.append(class_path+'/'+ image)
        image_id.append(int(path))
   # Chon ngau nhien 60 anh lam train,40 anh moi class lam anh test
    y=np.arange(100)
    np.random.shuffle(y)
    train_image_list+= [image_path[i] for i in y[0:60]]
    test_image_list+=[image_path[i] for i in y[60:100]]
    train_id+=[image_id[i] for i in y[0:60]]
    test_id+=[image_id[i] for i in y[60:100]]


In [14]:
# orb
# Initiate FAST detector
orb = cv2.ORB_create()
All_orb=[]# list contain all orb vectors from all training images:
for image in train_image_list:
    img = cv2.imread(image)

    kp, des = orb.detectAndCompute(img,None)
    for d in des:
        All_orb.append(d)

In [15]:
from matplotlib import pyplot as plt

In [16]:
from sklearn.cluster import MiniBatchKMeans

In [ ]:
##Thuc hien clustering tren BREIF-descriptors de tao codewords:
Number_code_words= 500
batch_size = 3000 # 3 * tong so anh=1000 
kmeans = MiniBatchKMeans(n_clusters=Number_code_words, batch_size=batch_size, verbose=1).fit(All_orb)

In [29]:
# Bieu dien tung anh thanh BOW
# Xap xi moi vector orb boi code word, sau do tinh histogram:


In [30]:
kmeans.verbose = False
train_img_bow = []
for image in train_image_list:
    img = cv2.imread(image)
    kp, des = orb.detectAndCompute(img,None)

    histo = np.zeros(Number_code_words) #
    nkp = np.size(kp)  #nkp: number of keypoints

    for d in des:
        idx = kmeans.predict([d])
        histo[idx] += 1
        
    histo=np.array(histo) # chuyen ve dang vector numpy
    histo=histo/nkp # normalized

    train_img_bow.append(histo)

In [31]:
# Test
kmeans.verbose = False
test_img_bow = []

for image in test_image_list:
    img = cv2.imread(image)
    kp, des = orb.detectAndCompute(img,None)
    histo = np.zeros(Number_code_words) 

    nkp = np.size(kp)  #nkp: number of keypoints
    for d in des:
      idx = kmeans.predict([d])
      histo[idx] += 1
        
    histo=np.array(histo) # chuyen ve dang vector numpy
    histo=histo/nkp # normalized

    test_img_bow.append(histo)



In [32]:
train_id = np.array(train_id)
test_id = np.array(test_id)

In [33]:
## Su dung SVM Classification

In [34]:
X = np.array(train_img_bow)
Testbow=np.array(test_img_bow)
Y = train_id

from sklearn.svm import SVC


classifier = SVC(gamma=30, C=10.0)
classifier.fit(X,Y)
res=classifier.predict(Testbow)

accuracy = sum(res==test_id)/len(test_id)
accuracy

0.4375

In [35]:
classifier = SVC(C=2.5,kernel='rbf',gamma='scale')
classifier.fit(X,Y)
res=classifier.predict(Testbow)
accuracy = sum(res==test_id)/len(test_id)
accuracy

0.445

In [36]:
classifier = SVC(C=2.5,kernel='poly',gamma='scale')
classifier.fit(X,Y)
res=classifier.predict(Testbow)
accuracy = sum(res==test_id)/len(test_id)
accuracy

0.44

In [37]:
classifier = SVC(C=2.5,kernel='linear',gamma='scale')
classifier.fit(X,Y)
res=classifier.predict(Testbow)
accuracy = sum(res==test_id)/len(test_id)
accuracy

0.345

In [38]:
classifier = SVC(C=2.5,kernel='sigmoid',gamma='scale')
classifier.fit(X,Y)
res=classifier.predict(Testbow)
accuracy = sum(res==test_id)/len(test_id)
accuracy

0.435

In [ ]:
# Thu cac detector, descriptor khac...
# kich thuoc tu dien khac nhau
# SVM +kernel khac nhau
#===> ACC???